In [1]:
#Convert similarity results into a similarity matrix format
import pandas as pd
columns = ["query_id", "subject_id", "identity", "alignment_length", "mismatches", "gap_opens", "q_start", "q_end", "s_start", "s_end", "e_value", "bit_score"]
blast_results = pd.read_csv("KIBA_Blast_result.txt", sep="\t", names=columns)
# Retrieve all unique protein IDs
protein_ids = sorted(set(blast_results["query_id"]).union(set(blast_results["subject_id"])))
# Initialize the similarity matrix
similarity_matrix = pd.DataFrame(0, index=protein_ids, columns=protein_ids, dtype=float)
for _, row in blast_results.iterrows():
    query_id = row["query_id"]
    subject_id = row["subject_id"]
    identity = row["identity"]
    similarity_matrix.loc[query_id, subject_id] = identity
    similarity_matrix.loc[subject_id, query_id] = identity  # 对称矩阵
# Save the similarity matrix to a CSV file
similarity_matrix.to_csv("KIBA_Blast_similarity_matrix.csv")

In [4]:

#Find the top 5 neighbors
file_path = 'KIBA_Blast_similarity_matrix.csv'  
df = pd.read_csv(file_path, index_col=0)  

def get_top_5_columns_excluding_diagonal(row):
    row_index = row.name
    row_series = pd.Series(row, index=df.columns)

    if row_index in df.columns:
        row_series.drop(row_index, inplace=True)

    top_5_columns = row_series.nlargest(5).index.tolist()

    while len(top_5_columns) < 5:
        top_5_columns.append(np.nan)
    
    return top_5_columns

top_5_columns_df = df.apply(get_top_5_columns_excluding_diagonal, axis=1, result_type='expand')

top_5_columns_df.columns = [f'Top_{i+1}' for i in range(5)]

# 打印结果
#file2_df = top_5_columns_df
#file2_df.set_index(file2_df.columns[0])
#file2_df = file2_df.loc[order_list]
#file2_df.to_csv("/home/rux/letter/KIBA/KIBA_Blast_top_5.csv",header = None)
# 保存到CSV文件
output_file_path = '/home/rux/letter/KIBA/KIBA_Blast_top_5.csv'
top_5_columns_df.to_csv(output_file_path, index=True,header=None)

output_file_path

'/home/rux/letter/KIBA/KIBA_Blast_top_5.csv'